### Dynamic HTML/ AJAX

![65](https://user-images.githubusercontent.com/38183218/42926013-0f316b96-8b6b-11e8-88d3-0c1bd1c4a869.PNG)

데이터 부분만 비동기로 가져오기에

개발자 도구에서 데이터가 보이지 않음

In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
resp = requests.get("http://example.webscraping.com/places/default/search")

In [ ]:
resp.text

In [ ]:
dom = BeautifulSoup(resp.content, "lxml")

In [ ]:
result = dom.select("#results")

In [ ]:
result #데이터부분이 비어있음

### Selenium

원래 목적은 autimating web app for testing purposes

우리는 브라우저를 직접 동작시켜 새롭게 바뀐 html문서를 dom으로 가져오는 데 쓴다

브라우저를 직접 동작시킨다는 것은 JavaScript를 이용해 비동기적으로 혹은 뒤늦게 불러와지는 컨텐츠들을 가져올 수 있다는 것이다. 

즉, AJAX 렌더딩 결과를 획득하는 것이며, '눈에 보이는' 컨텐츠라면 모두 가져올 수 있다.

우리가 requests에서 사용했던 .text의 경우 브라우저에서 ‘소스보기’를 한 것과 같이 동작하여, JS등을 통해 동적으로 DOM이 변화한 이후의 HTML을 보여주지 않는다. 

반면 Selenium은 실제 웹 브라우저가 동작하기 때문에 JS로 렌더링이 완료된 후의 DOM결과물에 접근이 가능하다.

In [2]:
from selenium import webdriver

Selenium은 webdriver라는 것을 통해 디바이스에 설치된 브라우저들을 제어할 수 있다. Chrome을 사용해 보자.

In [3]:
driver_path = 'chromedriver.exe'
driver =webdriver.Chrome(driver_path) #브라우저 받아야 동작

In [4]:
driver.get("http://example.webscraping.com/places/default/search")

In [5]:
obj = driver.find_element_by_id("search_term") #element를 찾아줌

In [6]:
obj.send_keys('korea')

In [8]:
driver.find_element_by_id("search").click()

In [9]:
driver.implicitly_wait(1)

In [62]:
driver.page_source[:1000] #str, 데이터를 가진 html을 리턴

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"><head>\n<meta charset="euc-kr" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\n<meta http-equiv="refresh" content="600" />\n<meta name="viewport" content="width=1106" />\n\n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n\n<meta property="og:title" content="네이버 뉴스" />\n<meta property="og:type" content="website" />\n<meta property="og:url" content="http://news.naver.com/main/home.nhn" />\n<meta property="og:image" content="https://ssl.pstatic.net/static.news/image/news/ogtag/navernews_200x200_20160804.png" />\n<meta property="og:description" content="정치, 경제, 사회, 생활/문화, 세계, IT/과학 등 언론사별, 분야별 뉴스 기사 제공" />\n\n<meta name="twitter:card" content="summary" />\n<meta name="twitter:title" content="네이버 뉴스" />\n<meta name="twitt

In [10]:
links = driver.find_element_by_css_selector("#results a")

In [15]:
links # not iterable

<selenium.webdriver.remote.webelement.WebElement (session="b860e038860bcacb95780410fa3855e1", element="0.0442578236922293-3")>

In [16]:
links = driver.find_elements_by_css_selector("#results a")
#elements 라는 메소드 써야 리스트로 가져옴

In [18]:
for link in links:
    print(link.text)

North Korea
South Korea


![70](https://user-images.githubusercontent.com/38183218/42926014-1001c9ee-8b6b-11e8-9c64-31d5d4b0e4b1.PNG)

#### 예제 네이버 메일 가져오기

json 객체로 아이디와 pwd를 저장하여 불러오자

In [33]:
import json
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

In [47]:
with open("account.json", 'r') as f:
    account = json.load(f)

In [48]:
account['id']

'tyui52003'

In [26]:
type(account)

dict

In [23]:
url = "https://www.naver.com"

In [24]:
driver_path = 'chromedriver.exe'
driver =webdriver.Chrome(driver_path)

In [25]:
driver.get(url)

In [32]:
driver.find_element_by_id('id').send_keys(account['id'])
driver.find_element_by_id('pw').send_keys(account['pwd'])

In [31]:
driver.find_element_by_css_selector(".btn_login > input").click()

네이버가 새로운 기기로 인식하여 기기 등록이 필요한 경우가 있다

이 상황을 위한 예외처리가 필요하다

In [ ]:
driver.find_element_by_css_selector(".btn_upload > a").click()#기기등록
driver.find_element_by_css_selector(".btn_maintain > a").click()#로그인유지

In [59]:
#함수로 만들어보자
def get_naver_mail(jsonFileName:str):
    with open(jsonFileName+".json", 'r') as f:
        account = json.load(f)
    
    driver_path = 'chromedriver.exe'
    driver =webdriver.Chrome(driver_path)
    driver.get("https://www.naver.com")
    
    driver.find_element_by_id('id').send_keys(account['id'])
    driver.find_element_by_id('pw').send_keys(account['pwd'])
    
    driver.find_element_by_css_selector(".btn_login > input").click()
    
    try: #기기등록 상황만을 예외로 간주
        driver.find_element_by_css_selector(".btn_upload > a").click()#기기등록
        driver.find_element_by_css_selector(".btn_maintain > a").click()#로그인유지
    
    except NoSuchElementException as e:
        pass
        
    finally:
        driver.get("https://mail.naver.com/?n=1531981389076&v=f") #메일에 접근
        html = BeautifulSoup(driver.page_source, "lxml")
        tmplst = html.select(".subject strong")
        
        for tmp in tmplst:
            print(tmp.contents[1])

In [61]:
get_naver_mail('accountNaver')

문의내용에 대하여 답변드립니다
주문하신 상품이 발송되었습니다
블로그의 새로운 변화 BLOSUM PROJECT를 시작합니다.
[엘롯데] 주문하신 상품이 발송되었습니다.
(광고) 사랑하는 이의 모습을 더 아름답게 보고 싶지 않으세요..?♥
[엘롯데] 주문이 완료되었습니다.
(광고)[온라인삼국지2] 신규서버 무신 그랜드 오픈!
[한국생산성본부]스포츠 아이디어 공모전 안내(창업 올림피아드) 안내 [~7.1 마감]
(광고) 미국 사립 학교도 CSIET 미국 인가 유학으로 가야 합니다!! 미국유학의 새로운 패러다임을 아시나요?? 지금 바로 무료 진학 컨설팅 받아보세요!@
[네이버페이]구매하신 제품의구매확정처리 부탁 드립니다
[SSG.COM] 백찬규 고객님께서 주문하신 상품내역입니다.
[네이버페이]주문하신 제품의 교환이 요청되었습니다
[SSG.COM] 백찬규 고객님께서 주문하신 상품내역입니다.
[네이버페이] 결제하신 내역을 안내해드립니다.
Re: 교수님 SE 과제1 컨펌부탁드립니다
